In [9]:
import pandas as pd

data = pd.read_csv('/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_gathered/all_human_annotations_processed.csv')


## choose random 100 samples
# data = data.sample(n=100, random_state=1)

In [10]:
data.columns

Index(['review_point', 'paper_id', 'venue', 'focused_review', 'actionability',
       'actionability_label', 'actionability_label_type', 'batch',
       'grounding_specificity', 'grounding_specificity_label',
       'grounding_specificity_label_type', 'verifiability',
       'verifiability_label', 'verifiability_label_type', 'helpfulness',
       'helpfulness_label', 'helpfulness_label_type', 'professional_tone',
       'professional_tone_label', 'professional_tone_label_type',
       'valid_point', 'valid_point_label', 'valid_point_label_type'],
      dtype='object')

In [11]:
aspects = [ 'actionability', 'grounding_specificity', 'verifiability', 'helpfulness']
samples_per_label = 20
# Initialize a dictionary to store sampled data for each aspect
sampled_data = {}

in_context_examples = {}

for aspect in aspects:
    # Define the relevant columns
    label_column = f"{aspect}_label"
    label_type_column = f"{aspect}_label_type"
    
    # Filter rows where label type is 'gold'
    aspect_gold_rows = data[data[label_type_column] == 'gold']
    
    # Group by the label and sample 105 rows per label value
    sampled_aspect = (
        aspect_gold_rows
        .groupby(label_column, group_keys=False)
        .apply(lambda group: group.sample(n=samples_per_label, random_state=42) 
               if len(group) >= samples_per_label else group)
    )

    ## 3 examples of each label to the in context examples,and the rest to the test data
    in_context_examples[aspect] = sampled_aspect.groupby(label_column).head(3)
    ### take only the focused_review, review_point and the label
    in_context_examples[aspect] = in_context_examples[aspect][['review_point', 'focused_review', f'{aspect}_label', aspect]]

    sampled_aspect = sampled_aspect.drop(in_context_examples[aspect].index)

    
    ## only keep the columns that are needed: review_point, paper_id, focused_review, and the aspect, and the label
    sampled_aspect = sampled_aspect[['review_point', 'paper_id', 'focused_review', label_column]]

    # Store the sampled data in the dictionary
    sampled_data[aspect] = sampled_aspect

for x in aspects:
    print(x, sampled_data[x].shape)
    ## save the df
    sampled_data[x].to_csv(f'/home/abdelrahman.sadallah/mbzuai/review_rewrite/chatgpt/test_data/{x}_test_data.csv', index=False)
# # Combine the sampled data for all aspects if needed
# final_sampled_df = pd.concat(sampled_data.values(), ignore_index=True)



actionability (47, 4)
grounding_specificity (49, 4)
verifiability (57, 4)
helpfulness (75, 4)


/tmp/ipykernel_3553157/1219742311.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.sample(n=samples_per_label, random_state=42)
/tmp/ipykernel_3553157/1219742311.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.sample(n=samples_per_label, random_state=42)
/tmp/ipykernel_3553157/1219742311.py:20: DeprecationWarning: DataFrameGroupBy.apply operated

In [12]:
## create excel sheet for the in context examples

with pd.ExcelWriter('/home/abdelrahman.sadallah/mbzuai/review_rewrite/chatgpt/test_data/in_context_examples.xlsx') as writer:
    for aspect in aspects:
        in_context_examples[aspect].to_excel(writer, sheet_name=aspect, index=False)